In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.classes import Operator, Coefficient, Term, Expression
from Modules.solver import *

In [3]:
hbar = Coefficient('hbar', order=0)
omega = Coefficient('omega', order=0)
Omega_z = Coefficient('Omega_z', order=0)
g = Coefficient('g', order=1)

a = Operator('a', subspace='oscillator', is_infinite=True, mat_representation=sp.Matrix([[0, 1, 0], [0, 0, 1], [0, 0, 0]]))
a_dag = Operator('{a^\\dagger}', subspace='oscillator', is_infinite=True, mat_representation=sp.Matrix([[0, 0, 0], [1, 0, 0], [0, 1, 0]]))

a.add_commutation_relation(a_dag, 1)
a_dag.add_commutation_relation(a, -1)

X = Operator('sigma_x', mat_representation=sp.Matrix([[0, 1], [1, 0]]), subspace='spin')
Z = Operator('sigma_z', mat_representation=sp.Matrix([[1, 0], [0, -1]]), subspace='spin')
Y = Operator('tau_y', mat_representation=sp.Matrix([[0, -1j], [1j, 0]]), subspace='charge')

H = hbar * omega * a_dag * a + hbar * Omega_z * 0.5 * Z + hbar * g * (a + a_dag) * X
H


hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar a sigma_x + g*hbar {a^\dagger} sigma_x

In [4]:
H_new, subspaces = H.domain_expansion()

In [14]:
Vk = H_new.group_by_diagonal()[False].group_by_order()[1]
Vk
get_ansatz(Vk)[0]

\varepsilon_{(1)} S^{(0)}_1 + \varepsilon_{(1)} S^{(1)}_1 a + \varepsilon_{(1)} S^{(2)}_1 {a^\dagger}

In [ ]:
subspaces

['spin']

In [ ]:
H_new

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar sigma_x a + g*hbar sigma_x {a^\dagger}

In [ ]:
H_new.terms[0].info

{'coeff': [hbar, omega, 1, 1, 1, 1, 1, 1], 'oscillator': [{a^\dagger}, a]}

In [ ]:
S, symbols_for_s = get_ansatz(H_new)
S.terms[0].info

{'coeff': [1, \varepsilon^{(1)}], 'finite': [S^{(0)}]}

In [ ]:
for key, value in ((H_new.group_by_order()[0] | S ) + H_new.group_by_order()[1].group_by_diagonal()[False]).group_by_infinite_terms().items():
    print(f'\n\n\n{key}:\n')
    print([sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) for term in value.terms])
    mats = sp.Add(*[sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) * term.mat_representation["finite"] for term in value.terms])
    display(mats)
    display(sp.solve(mats, symbols_for_s))




1:

[-Omega_z*hbar/2, Omega_z*hbar/2]


Matrix([
[                      0, Omega_z*S^{(0)}_1*hbar],
[-Omega_z*S^{(0)}_2*hbar,                      0]])

{S^{(0)}_1: 0, S^{(0)}_2: 0}




1 a:

[-hbar*omega, -Omega_z*hbar/2, Omega_z*hbar/2, g*hbar]


Matrix([
[                                  -S^{(1)}_0*hbar*omega, Omega_z*S^{(1)}_1*hbar - S^{(1)}_1*hbar*omega + g*hbar],
[-Omega_z*S^{(1)}_2*hbar - S^{(1)}_2*hbar*omega + g*hbar,                                  -S^{(1)}_3*hbar*omega]])

{S^{(1)}_0: 0,
 S^{(1)}_1: -g/(Omega_z - omega),
 S^{(1)}_2: g/(Omega_z + omega),
 S^{(1)}_3: 0}




1 {a^\dagger}:

[hbar*omega, -Omega_z*hbar/2, Omega_z*hbar/2, g*hbar]


Matrix([
[                                   S^{(2)}_0*hbar*omega, Omega_z*S^{(2)}_1*hbar + S^{(2)}_1*hbar*omega + g*hbar],
[-Omega_z*S^{(2)}_2*hbar + S^{(2)}_2*hbar*omega + g*hbar,                                   S^{(2)}_3*hbar*omega]])

{S^{(2)}_0: 0,
 S^{(2)}_1: -g/(Omega_z + omega),
 S^{(2)}_2: g/(Omega_z - omega),
 S^{(2)}_3: 0}

In [37]:
from IPython.display import display, Math
def solver(H, order):
    H_expanded = H.domain_expansion()[0]
    H_0 = H_expanded.group_by_order()[0]
    H_final = deepcopy(H_0)
    Bk = 0

    S = 0

    for k in range(1, order+1):
        print(f"Order {k}")
        H_below_k = sum(H_expanded.group_by_order().get(j, 0) for j in range(k+1))
        Vk = H_below_k.group_by_diagonal()[False].group_by_order().get(k, 0)
        S_k, symbols_sk = get_ansatz(Vk + Bk)
        solution_sk = dict(zip(symbols_sk, [0 for _ in symbols_sk]))
        #display(Bk)
        for key, value in ((H_0 | S_k) + (Vk + Bk)).group_by_infinite_terms().items():
            print("Im here 1")
            mats = sp.Add(*[sp.Mul(*map(lambda x: x.representation , term.info["coeff"])) * term.mat_representation["finite"] for term in value.terms])
            solution_sk.update(sp.solve(mats, symbols_sk))
        
        S_k_solved = 0
        for term in S_k.terms:
            print("Im here 2")
            term.mat_representation["finite"] = term.mat_representation["finite"].subs(solution_sk)
            if sum(sp.Abs(term.mat_representation["finite"])) == 0:
                continue
            S_k_solved += term
        S += S_k_solved
        display(S)
        tmp_H = float(1 / sp.factorial(k)) * H_below_k.nested_commutator(S, k)
        H_final += tmp_H.group_by_diagonal().get(True, Expression(Term(Coefficient(0)))).group_by_order().get(k, 0)
        display(H_final)
        Bk = tmp_H.group_by_diagonal()[False].group_by_order()[k+1]
    
    return H_final


In [38]:
H_expanded = H.domain_expansion()[0]
H_expanded.group_by_order()[0]

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z

In [39]:
H

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar a sigma_x + g*hbar {a^\dagger} sigma_x

In [40]:
solver(H, 2)

Order 1
Im here 1
Im here 1
Im here 1
Im here 2
Im here 2
Im here 2


\varepsilon_{(1)} S^{(1)}_1 a + \varepsilon_{(1)} S^{(2)}_1 {a^\dagger}

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z

Order 2
Im here 1
Im here 1
Im here 1
Im here 1
Im here 1
Im here 2
Im here 2
Im here 2
Im here 2
Im here 2


\varepsilon_{(1)} S^{(1)}_1 a + \varepsilon_{(1)} S^{(2)}_1 {a^\dagger} + \varepsilon_{(1)} S^{(0)}_2 + \varepsilon_{(2)} S^{(1)}_2 a**2 + \varepsilon_{(2)} S^{(2)}_2 a*{a^\dagger} + \varepsilon_{(2)} S^{(3)}_2 {a^\dagger}*a + \varepsilon_{(2)} S^{(4)}_2 {a^\dagger}**2

KeyboardInterrupt: 

In [25]:
g * a | a_dag

g